# Testing of robustness against Fast Gradient Sign Method (FGSM)

### White box environment with TensorFlow framework

### Importing required packages

In [16]:
from __future__ import absolute_import, division, print_function, unicode_literals

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
import tensorflow as tf


from art.attacks import FastGradientMethod
from art.classifiers import KerasClassifier
from art.utils import load_mnist

tf.compat.v1.disable_eager_execution()

### Load Dataset

In [17]:
# Step 1: Load the MNIST dataset

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

### Creating a sample model
A sample model is used to demonstrate the use of ART library
Model to be tested will be replaced here

In [18]:
# Step 2: Create the model

model = Sequential()

model.add(Conv2D(4,5,padding='same',input_shape = [28, 28, 1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(10,5, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Wrapping of ART classifier
Wrapping is necessary to use ART functions

In [19]:
# Step 3: wrap ART tf classifier
classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value))
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)

Epoch 1/10
469/468 [==============================] - 5s 10ms/step - loss: 0.3308 - accuracy: 0.9043
Epoch 2/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0973 - accuracy: 0.9715
Epoch 3/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0737 - accuracy: 0.9773
Epoch 4/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0572 - accuracy: 0.9824
Epoch 5/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0470 - accuracy: 0.9849
Epoch 6/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0428 - accuracy: 0.9870
Epoch 7/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0346 - accuracy: 0.9894
Epoch 8/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0339 - accuracy: 0.9899
Epoch 9/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0273 - accuracy: 0.9917
Epoch 10/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0253 - accura

### Evaluating model based on original Dataset

In [20]:
# Step 5: Evaluate the ART classifier on benign test examples

predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on benign test examples: {}%'.format(accuracy * 100))

Accuracy on benign test examples: 98.72%


### Creating Adversarial samples
Using Fast Gradient method

In [21]:
# Step 6: Generate adversarial test examples
attack = FastGradientMethod(classifier=classifier, eps=0.2)
x_test_adv = attack.generate(x=x_test)

### Evaluating model on adversarial samples

In [22]:
# Step 7: Evaluate the ART classifier on adversarial test examples

predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))

Accuracy on adversarial test examples: 21.78%


### Evaluation results
- Using accuracy as an evaluation metric, we can see that the sample model's performance dropped by a huge amount after going through adversarial samples.
- Results is normal as sample model does not have defence against adversarial attacks


## Adding defence layers based off advesarial attack
we will now add basing adversarial training defence to the model and evaluate its robustness using accuracy

### Creating more data for adversarial training
as adversarial datasets from x_train will be used to evaluate the trained model, we will need to create more data for adversarial training of model.
Thus, we will use Keras image augmentation to create more data for adversarial training.

In [23]:
#importing packages
from art.defences import AdversarialTrainer
from keras.preprocessing.image import ImageDataGenerator
from art.data_generators import KerasDataGenerator

In [24]:
# Build a Keras image augmentation object and wrap it in ART
batch_size = 50
datagen = ImageDataGenerator(horizontal_flip=True, width_shift_range=0.125, height_shift_range=0.125,
                             fill_mode='constant', cval=0.)
datagen.fit(x_train)
art_datagen = KerasDataGenerator(datagen.flow(x=x_train, y=y_train, batch_size=batch_size, shuffle=True),
                                 size=x_train.shape[0], batch_size=batch_size)


### Creating and training adversarial trainer

In [25]:
# Create adversarial trainer and perform adversarial training
adv_trainer = AdversarialTrainer(classifier, attacks=attack, ratio=1.)
adv_trainer.fit_generator(art_datagen, nb_epochs=2)

Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 11.6202 - accuracy: 0.0400
Epoch 1/1
1/1 [==============================] - 0s 15ms/step - loss: 12.4008 - accuracy: 0.1000
Epoch 1/1
1/1 [==============================] - 0s 16ms/step - loss: 11.8836 - accuracy: 0.0400
Epoch 1/1
1/1 [==============================] - 0s 17ms/step - loss: 7.5087 - accuracy: 0.0400
Epoch 1/1
1/1 [==============================] - 0s 16ms/step - loss: 7.3179 - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 16ms/step - loss: 5.5617 - accuracy: 0.0400
Epoch 1/1
1/1 [==============================] - 0s 14ms/step - loss: 3.2880 - accuracy: 0.0400
Epoch 1/1
1/1 [==============================] - 0s 15ms/step - loss: 3.4371 - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 15ms/step - loss: 2.8944 - accuracy: 0.0400
Epoch 1/1
1/1 [==============================] - 0s 16ms/step - loss: 2.6682 - accuracy: 0.0400
Epoch 1/1
1/1 [===============

In [26]:
# Step 7: Evaluate the ART classifier on adversarial test examples

predictions_adv = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions_adv, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))

Accuracy on adversarial test examples: 32.879999999999995%


### Evaluation results
- UAs we can see, the model performed better in terms of accuracy when it is adversarially trained against adversarial attacks
